In [1]:
# A IMPORTER

import pandas as pd
import json
import unidecode
import re
import numpy as np

## Cleaning the data

On charge notre dataset. Puis on créé un autre dataset reprenant les features qui nous intéressent pour appliquer nos processus NLP.

In [2]:
data = pd.read_json('indeed6.json', encoding = 'utf-8', lines=True)

In [3]:
job_to_class = data[['titre', 'texte']]

Ici, nous avons décidé d'utiliser la libraire spaCy. C'est elle qui nous permettra de parseriser nos descriptions et nos titres, retirer les stop words, et tokenizer.

In [4]:
import spacy 
#Chargeons notre vocabulaire spaCy
nlp_fr = spacy.load("fr_core_news_sm")

In [5]:
#Chargeons nos stopwords qu'on mettra dans une liste
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.fr.stop_words import STOP_WORDS
french_stopwords = list(spacy.lang.fr.stop_words.STOP_WORDS)
english_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
stop_words = french_stopwords + english_stopwords

#J'importe la ponctuation qui nous servira au traitement du texte
import string
punctuations = string.punctuation

#Ainsi que notre Parser qui comprend la structure grammatical de notre texte
from spacy.lang.fr import French
parser = French()

Dans un premier temps, on avait opté pour utiliser strictement spaCy pour le traitement global. Toutefois, il restait imparfait avec beaucoup de "déchet" dans nos tokens.
On choisit donc de subdiviser à nouveau notre traitement. 
Tout d'abord on retire les caractères indésirables. Puis, dans la mesure où on a déjà créé des nouvelles features grâce au regex, on supprime ces éléments de nos données afin d'éviter tout risque de colinéarité dans l'entraînement de nos modèles de machine learning.
Enfin on tokenize nos textes et on nettoie à nouveau les caractères indésirables qui ont pu se glisser dedans.

In [6]:
from string import digits

#Ces caractères indésirables ne nous fournissent pas d'informations pertinentes. On les retire
def clean_and_lower(string):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", "\n", 
                "...", "/","'"]
    for car in forbid_car:
            cleaned_string = string.replace(car, ' ')
    return cleaned_string

#On retire les termes déjà étudiés lors de nos regex, et qui sont déjà des features
def terme_a_retirer(string):
        string = re.sub(pattern = '(jeune[es().]* diplôme[es().]*|junior[s]*|novice[s]*| débutant[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(expérimenté[es().]*|senior[s]*|confirmé[es().]*)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(licence|bac[ +]*[23/]+|iut|dut|bts| bsc|master|bac[ +]*5|ingénieur[.()es]*|grande[s]* école[s]*| msc|doctorat[s]*|docteur[.()es]*|ph[.]*d|thèse[s]*|bac)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(minimum [1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* d\'expérience|[1-9]*[/aou-]*\d+ an[nées]* minimum d\'expérience|expérience minimum de *[1-9]*[/aou-]*\d+ an[nees]|expérience|d\'expérience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(nantes|bordeaux|paris|île-de-france|lyon|toulouse)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+ ]*[ \-k€]*[\d+ ]*\d+,?\d+[ ]*[k€$]+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(cdi|CDI|Cdi|cdd|CDD|Cdd|stage[s]*|stagiaire[s]*|intern[s]*|internship[s]*|freelance|Freelance|FREELANCE||indépendant[s]*|par an|/an|temps plein|3dexperience)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(h[/ \-]*f|f[/ \-]*h)', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '(salaire|[\d+]+[ ]*an[nées]*|[\d+]+[ ]*jour[s]|/mois|/jour[s]*|/semaine[s]*|[\d+]+ mois|[\d+]+ semaine[s]*|mois|semaine[s]*|jour[s]*])', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\d+]+[]*[èe]*[mes]*', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = r'\d+', repl = '', string = string, flags=re.IGNORECASE)
        string = re.sub(pattern = '[\w\.-]+@[\w\.-]+', repl = '', string = string, flags=re.IGNORECASE)
        return string

#On tokenize notre texte, on retire les pronoms, stop words, et on réduit chaque terme à sa racine
def tokenize(string):
    parsed_string = parser(string)
    tokenized_string = [word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in parsed_string]
    cleaned_tokenized_string = [word for word in tokenized_string if word not in punctuations and word not in stop_words]
    return cleaned_tokenized_string

#On retire le substrat de ponctuaction indésirable qui peut rester dans nos tokens
def strip_final_punctuation(list_of_tokens):
    forbid_car= [":", ";", ",", "&", "(", ")",
                '"', "!", "?", "*", "-", 
                "...", "/","'", "\\", "·", ".", "∙", "•"
                "–"]
    
    for i in range(len(list_of_tokens)):
        for car in forbid_car:
            list_of_tokens[i] = list_of_tokens[i].replace(car, '')
    return list_of_tokens
    


On définit ensuite une fonction finale de nettoyage qui tournera sur nos descriptions et sur nos titres.

In [7]:
def cleaning(string):
    cleaned_string = clean_and_lower(string)
    cleaned_string = terme_a_retirer(cleaned_string)
    tokenized_string = tokenize(cleaned_string)
    tokenized_string = strip_final_punctuation(tokenized_string)
    return tokenized_string
    

## Traitement des descriptions avec SpaCy

### I. Tokenize, clean, lemmatize

In [8]:
#On tokenize complètement avec spaCy nos descriptions

job_to_class['spacy_description'] = [cleaning(job_to_class.loc[i, 'texte']) for i in range(len(job_to_class['texte']))]

C:\Users\David\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### II. Create the TF-IDF term-documents matrix with the processed data

Il nous faut maintenant évaluer le poid relatif de chaque terme dans nos descriptions. On choisit donc de vectoriser en utilisant une matrice Tf-Idf.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

L'initialisation des paramètres de notre vectorisation reste encore une question ouverte. L'idéal aurait été de tester automatiquement plusieurs nombres maximaux de features et plusieurs n_gram, stocker les résultats prédictifs dans un log, et choisir le plus adapté. Faute de temps, on a choisi un grand nombre de features correspondant à la grande variété de nos annonces.

In [10]:
#On initialise nos paramètres pour la vectorisation

max_features_description = 5000
min_n_description = 1
max_n_description = 3

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_description = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_description, sublinear_tf = True,
                                    ngram_range=(min_n_description, max_n_description))


tfidf_matrix_description = tfidf_vectorizer_description.fit_transform(job_to_class['spacy_description'])


### III. Get the features

On utilise maintenant notre matrice term-document pour extraire les features les plus importantes de nos descriptions. C'est celles-ci qui vont ensuite nous servir pour l'extraction de topic.

In [11]:
features_description = tfidf_vectorizer_description.get_feature_names()

In [12]:
print(features_description)

['', ' profil', ' yu', ' yu continuer', '7e', '7e externe', 'ability', 'ability work', 'able', 'aborder', 'accenture', 'accepter', 'access', 'accessible', 'accompagnement', 'accompagnement client', 'accompagnement client dan', 'accompagner', 'accompagner client', 'accompagner client dan', 'accompagner croissance', 'accompagner dan', 'accompagner dan évolution', 'accompagner développement', 'accompagner entreprise', 'accompagner quotidien', 'accompagner transformation', 'accompagner équiper', 'accompagner évolution', 'accomplir', 'accomplish', 'accomplish extraordinary', 'accomplish extraordinary ground', 'accord', 'according', 'account', 'account manager', 'accounting', 'accounts', 'accroître', 'accueil', 'accueillir', 'accès', 'accélération', 'accélérer', 'accélérer cgi', 'accélérer cgi plan', 'achat', 'achats', 'achieve', 'achieve excellence', 'achieve excellence diversity', 'achieving', 'achieving workforce', 'achieving workforce diversity', 'acquis', 'acquisition', 'acquérir', 'act

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups


Une fois nos features extraites, on choisit d'utiliser trois méthodes pour extraire les topics: Nonnegative Matrix Factorisation, Latent Dirichlet Allocation et Latent Semantic Index.
On créé nos trois matrices d'extraction.

In [13]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [14]:
lda_description = LatentDirichletAllocation(n_components= 10, max_iter=10, learning_method='online',verbose=True)
data_lda_description = lda_description.fit_transform(tfidf_matrix_description)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [15]:
# Non-Negative Matrix Factorization Model
nmf_description = NMF(n_components = 10)
data_nmf_description = nmf_description.fit_transform(tfidf_matrix_description)

In [16]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_description = TruncatedSVD(n_components = 10)
data_lsi_description = lsi_description.fit_transform(tfidf_matrix_description)

On imprime nos divers topics pour observer s'il y a de la cohérence et si c'est exploitable.

In [17]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_description.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [18]:
print("LDA_description Model:")
selected_topics(lda_description)

LDA_description Model:
Topic 0:
[('ingeniance', 35.29996754635082), ('marcher financier', 19.56849629118027), ('philosophie devops', 19.29318435156892), ('philosophie', 18.81381370670896), ('offrir expertiser', 18.673188989001634), ('disruptives', 18.555846749575807), ('dynamique stimuler', 18.140989718829214), ('devops permettre', 18.089315453474363), ('spécialiste secteur', 18.051605606166834), ('technologie disruptives', 17.984355675535763)]
Topic 1:
[('team', 201.2792087925192), ('experience', 189.32550889382665), ('work', 162.04668684649278), ('skills', 154.168022814454), ('haver', 146.28503311002297), ('new', 124.63127172896893), ('knowledge', 116.58490612372654), ('development', 111.46233187916172), ('working', 110.3854612133674), ('company', 109.06728584253109)]
Topic 2:
[('situation handicap', 37.864850154520624), ('handicap', 36.07108346307456), ('ouvrir situation', 31.936625100237897), ('ouvrir situation handicap', 31.802939641816945), ('poster ouvrir situation', 31.60599209

In [19]:
print("LSI_description Model:")
selected_topics(lsi_description)

LSI_description Model:
Topic 0:
[('dan', 0.13606636235333158), ('d', 0.12989700923512798), ('équiper', 0.11942025037979885), ('l', 0.11490740576679583), ('développement', 0.11364575163274795), ('projet', 0.11133192736737042), ('client', 0.1089839363634034), ('technique', 0.10312436092379243), ('plaire', 0.08905811204291472), ('rechercher', 0.08574273829525851)]
Topic 1:
[('team', 0.18095150520551856), ('experience', 0.16700454218188976), ('work', 0.14263456506367436), ('skills', 0.1368329046621387), ('haver', 0.12616250524435818), ('new', 0.10709989548853635), ('dater', 0.09907564098331394), ('development', 0.09881757794896057), ('strong', 0.09879184418450135), ('knowledge', 0.0986136900754979)]
Topic 2:
[('activus', 0.09374168617017897), ('conseil', 0.07544183116252659), ('proximité', 0.0753321747675446), ('consultant', 0.07269873676700296), ('savoir accompagner', 0.07006420752312008), ('recruter', 0.06961028123717114), ('perspective', 0.0651116451725764), ('renforcer', 0.064468785716

In [20]:
print("NMF_description Model:")
selected_topics(nmf_description)

NMF_description Model:
Topic 0:
[('système', 0.7368519406403398), ('devops', 0.6781194799675562), ('infrastructure', 0.6403843785436413), ('technique', 0.5673513485993336), ('cloud', 0.5591693715285092), ('déploiement', 0.5320712719173657), ('production', 0.5217611732718901), ('linux', 0.5037507540710421), ('sécurité', 0.48459900302500636), ('outil', 0.46574604147984877)]
Topic 1:
[('team', 0.8509512843385956), ('experience', 0.7699343172067843), ('work', 0.6462520680659418), ('skills', 0.6135353484379356), ('haver', 0.5748381519125022), ('new', 0.49970724198634564), ('dater', 0.47792402618046603), ('strong', 0.4630701790045548), ('business', 0.4541436514984149), ('development', 0.4503183751383363)]
Topic 2:
[('activus', 0.3796139061288371), ('proximité', 0.2917957853189344), ('savoir accompagner', 0.284550385101519), ('recruter', 0.2807737232764991), ('perspective', 0.25834101109243235), ('renforcer', 0.25293820050280275), ('recruter consultant', 0.2519201942505969), ('local ouvrir', 

## Traitement des titres avec SpaCy

Ce qui a été obtenu via le traitement des descriptions n'est finalement pas satisfaisant. On essaye ici de concentrer le traitement sur les titres de chaque annonce. L'intuition c'est que l'information pour clusteriser nos annonces est plus concentrée sur les titres. Et donc la possibilité de différenciation sera plus importante.

Les étapes étant les mêmes que pour la description, on ne répétera pas les commentaires précédents. On se contentera d'analyser les résultats.

### I. Tokenize, clean, lemmatize

In [21]:
job_to_class['spacy_titre'] = [cleaning(job_to_class.loc[i, 'titre']) for i in range(len(job_to_class['titre']))]

C:\Users\David\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### II. Create the TF-IDF term-documents matrix with the processed data

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Ici on a fait le choix de réduire les max_features à 500, et le n_gram à 1. En effet, les titres sont bien plus court, et bien plus redondants. Mais là aussi il aurait fallu tester automatiquement plusieurs valeurs et sélectionner celles qui donnent les meilleures métriques.

In [23]:
#On initialise nos paramètres pour la vectorisation

max_features_titres = 500
min_n_titres = 1
max_n_titres = 1

#On crée une fonction inutile. En effet TfidfVectorizer ne peux pas gérer des documents déjà tokenisés

def dummy_fun(doc): 
    return doc

#On vectorise

tfidf_vectorizer_titres = TfidfVectorizer( analyzer = 'word',
                                    tokenizer = dummy_fun,
                                    preprocessor = dummy_fun,
                                    token_pattern = None,
                                    max_df = 1.0, min_df = 1,
                                    max_features = max_features_titres, sublinear_tf = True,
                                    ngram_range=(min_n_titres, max_n_titres))


tfidf_matrix_titres = tfidf_vectorizer_titres.fit_transform(job_to_class['spacy_titre'])


### III. Get the features !

In [24]:
features_titres = tfidf_vectorizer_titres.get_feature_names()
features_titres

['',
 '7e',
 '@pwc',
 'abap',
 'account',
 'achats',
 'acquisition',
 'actuaire',
 'administrateur',
 'administrator',
 'advisory',
 'affairer',
 'affaires',
 'agence',
 'agencer',
 'agile',
 'al',
 'alternance',
 'alternant',
 'alternante',
 'amoa',
 'analyse',
 'analyst',
 'analyste',
 'analytics',
 'android',
 'anglais',
 'angular',
 'angularjs',
 'api',
 'applicatif',
 'application',
 'applications',
 'apprentissage',
 'architect',
 'architecte',
 'architecture',
 'artificielle',
 'aspnet',
 'assistant',
 'assistante',
 'associate',
 'assurance',
 'ational',
 'audit',
 'auditeur',
 'automation',
 'automatisation',
 'automobile',
 'avantvente',
 'aws',
 'axa',
 'azure',
 'aéronautique',
 'b',
 'bancaire',
 'banque',
 'base',
 'based',
 'baser',
 'bb',
 'bi',
 'big',
 'bigdata',
 'btob',
 'business',
 'bw',
 'c',
 'c#net',
 'campaign',
 'center',
 'centre',
 'cfo',
 'chain',
 'change',
 'chantier',
 'charge',
 'charger',
 'chargé',
 'chargée',
 'chef',
 'chief',
 'client',
 'clients'

### IV. Find the (features x topics) matric with the LDA/LSI/NMF and generate groups

In [25]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [26]:
lda_titres = LatentDirichletAllocation(n_components= 20, max_iter=10, learning_method='online',verbose=True)
data_lda_titres = lda_titres.fit_transform(tfidf_matrix_titres)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [27]:
df_topic_20 = pd.DataFrame(data_lda_titres)
df_topic_20
df_topic_20.to_csv("topic_20.csv")

In [28]:
# Non-Negative Matrix Factorization Model
nmf_titres = NMF(n_components = 20)
data_nmf_titres = nmf_titres.fit_transform(tfidf_matrix_titres)

In [29]:
# Latent Semantic Indexing Model using Truncated SVD
lsi_titres = TruncatedSVD(n_components = 15)
data_lsi_titres = lsi_titres.fit_transform(tfidf_matrix_titres)

In [30]:
# Functions for printing keywords for each topic
def selected_topics(model, top_n = 15):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(tfidf_vectorizer_titres.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [31]:
# print("LDA_titres Model:")
# type(selected_topics(lda_titres))

In [32]:
# print("LSI_titres Model:")
# selected_topics(lsi_titres)

In [33]:
print("NMF_titres Model:")
print(type(selected_topics(nmf_titres)))

NMF_titres Model:
Topic 0:
[('développeur', 6.613723921255166), ('python', 0.47187422802662354), ('informatique', 0.27569399914770815), ('android', 0.2402352766144332), ('concepteur', 0.21786707277415807), ('angular', 0.17965300361048692), ('logiciel', 0.15402097854187374), ('react', 0.1533283290110522), ('ios', 0.15041125293141897), ('mobile', 0.14576397152536333), ('analyste', 0.13995800328101587), ('javascript', 0.1249186185321554), ('sql', 0.12251992301374998), ('kend', 0.1146747538015386), ('salesforce', 0.10455420450290369)]
Topic 1:
[('data', 5.020450804748571), ('scientist', 2.379322645628658), ('big', 1.1626693784775906), ('science', 0.21240493482678555), ('lead', 0.15721913866607398), ('architect', 0.13609377063980002), ('–', 0.12387151583028923), ('analyste', 0.10312553535359419), ('analytics', 0.09846960221238801), ('tech', 0.08157267635601964), ('ingenieur', 0.060972996747158024), ('consultante', 0.060730978123845175), ('protection', 0.04597257558901363), ('intelligence', 

[('développement', 3.7272555494363093), ('d', 1.3563838309182665), ('responsable', 1.0739733819672623), ('étude', 0.9903560556826464), ('logiciel', 0.6160889238742749), ('informatique', 0.4415495393017452), ('python', 0.3957843578880051), ('etudes', 0.3629531510820918), ('commercial', 0.34092831764928455), ('chargé', 0.2865107938136502), ('–', 0.231539037308511), ('système', 0.19824108769951768), ('application', 0.19627243336177946), ('etude', 0.18404242000286494), ('affairer', 0.18088582726095298)]
Topic 17:
[('net', 4.0001122397463345), ('lead', 0.4240617440530363), ('core', 0.17703667328857736), ('tech', 0.12279831529254596), ('azure', 0.11712820412278814), ('angular', 0.08187634446103634), ('développeurs', 0.06369904317570935), ('wpf', 0.048640459008732435), ('développeur', 0.04206491197986572), ('expert', 0.040952255809830876), ('editeur', 0.040022811961812906), ('sharepoint', 0.0343827682206642), ('developer', 0.03265517516145216), ('technique', 0.03196529638857191), ('sénior', 0

In [34]:
df_topic_20 = pd.DataFrame(data_nmf_titres)
df_topic_20.to_csv("topic_20.csv")

In [35]:
df_topic_20.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000,19418.000000
mean,0.014222,0.008506,0.007348,0.007135,0.008020,0.006778,0.008157,0.005174,0.003231,0.002737,0.005919,0.004775,0.005941,0.004161,0.006433,0.004704,0.006976,0.004725,0.004347,0.006139
std,0.027039,0.026404,0.032176,0.025587,0.030188,0.029134,0.028401,0.025069,0.015293,0.016234,0.026068,0.024399,0.023860,0.021509,0.026637,0.026134,0.021842,0.025971,0.022317,0.028920
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.026478,0.000169,0.000163,0.000937,0.000048,0.000794,0.000888,0.000456,0.000047,0.000014,0.000583,0.000084,0.000519,0.000000,0.000263,0.000010,0.002586,0.000058,0.000000,0.000386
max,0.148776,0.164727,0.207754,0.188958,0.213054,0.220268,0.212021,0.215250,0.134009,0.147971,0.213612,0.205115,0.201517,0.173223,0.222552,0.249181,0.191681,0.245821,0.216327,0.270855


## Conclusion

L'application de notre méthode de NLP sur les titres s'avère donner des extractions de topic plus cohérent que celle sur les descriptions. Toutefois l'ajout de notre matrice NMF dans les features de notre dataset nous fait gagner une accuracy relativement marginale. Et malheureusement les topics ne sont pas suffisamment cohérent pour alimenter la data analysis côté client.
Peut-être que la libraire Bert pourrait s'avérer plus utile sur ces deux niveaux.